# PROJET FINAL DE MACHINE LEARNING

# CONCEPTION D'UN CHATBOT POUR LA GESTION DES STAGIAIRES DANS UNE ENTREPRISE

## IMPORTATION DES BIBLIOTHEQUES

In [2]:
import json
import numpy as np
import pandas as pd
import random
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
# import tkinter as tk
# from tkinter import *

## CHARGEMENT DES DONNEES

Les données sont chargées depuis le fichier JSON chatbot_data.json qui contient les intents avec les patterns et les tags.

In [3]:
# Charger les données
with open('intents.json', 'r', encoding='utf-8') as file:
    data = json.load(file)

In [4]:
# Convertir le JSON en DataFrame
df = pd.DataFrame(data['intents'])

# Afficher le DataFrame dans le notebook
df

,tag,patterns,responses
0,salutations,"[bonjour, salut, hey, coucou, hello, yo, Bonjo...","[Salut !, Hey ! Comment puis-je vous aider ?, ..."
1,annéeCréation,"[Quand cette entreprise a-t-elle été fondée ?,...","[Elle a été créée en 2017., Cette entreprise a..."
2,applicationsDéveloppées,[Quelles applications ont été développées par ...,[Les produits logiciels déjà créés par l'entre...
3,fondateur,"[Qui a fondé cette entreprise ?, Qui est le cr...",[Ngeufack Thierry est le créateur de l'entrepr...
4,domaineActivité,[Quelle est l'activité principale de cette ent...,[Le domaine d'activité de l'entreprise est le ...
5,nombreEmployé,[Quel est le personnel actuel de l'entreprise ...,[L'entreprise emploie actuellement 8 personnes...
6,nomEntreprise,"[Quel est le nom de l'entreprise ?, Comment s'...","[L'entreprise a pour nom Boost Compagny Inc., ..."
7,auRevoir,"[au revoir, à bientôt, à la prochaine, à plus ...","[Au revoir ! Si vous avez d'autres questions, ..."
8,remerciements,"[merci, merci pour votre aide, merci beaucoup,...",[De rien ! Si vous avez besoin d'aide à l'aven...
9,invalide,"[, yayayya, fzrgzrrgzr, hbvcjhdbvc, vgjvj, cul]","[Je suis désolé, je n'ai pas bien compris. Pou..."


## PREPARATION DES DONNEES

Les patterns (questions) sont extraits dans X et les tags (intentions) dans y.

In [5]:
# Extraire les intents, questions et réponses
intents = data['intents']
patterns = []
tags = []
responses = {}

for intent in intents:
    for pattern in intent['patterns']:
        patterns.append(pattern)
        tags.append(intent['tag'])
    responses[intent['tag']] = intent['responses']

# Transformation des intentions en nombres
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(tags)

In [9]:
patterns

['bonjour',
 'salut',
 'hey',
 'coucou',
 'hello',
 'yo',
 'Bonjour',
 'Bonne journée',
 'Bon matin',
 'Bon après-midi',
 'Bonsoir',
 'Bonne journée à vous',
 'Enchanté',
 'Comment allez-vous ?',
 'Coucou',
 'Yo',
 'Hey',
 'Allo',
 'Ça va ?',
 'Salut les amis',
 'Hé, salut !',
 'Quoi de neuf ?',
 'Tiens',
 'Salutations',
 'Salut tout le monde',
 'Bonjour à tous',
 'Bonjour tout le monde',
 'Salut à tous',
 'Hello tout le monde',
 'Bon matin à tous',
 'Bonsoir à tous',
 'Coucou tout le monde',
 'Hey tout le monde',
 'Bonjour les amis',
 'Salut les gens',
 'Hello les amis',
 'Hey les amis',
 'Salut la compagnie',
 'Bonjour la compagnie',
 'Coucou la compagnie',
 'Hello la compagnie',
 'Hey la compagnie',
 'Salut, ça va ?',
 'Bonjour, comment ça va ?',
 'Salut, quoi de neuf ?',
 'Hello, comment allez-vous ?',
 'Hey, ça va ?',
 'Salut, comment ça roule ?',
 'Bonjour, ça fait plaisir de vous voir',
 'Hello, ça fait plaisir de vous voir',
 'Salut, comment vous sentez-vous ?',
 'Bonjour, comm

## PIPELINE SCIKIT-LEARN

Un pipeline est créé avec CountVectorizer pour convertir les textes en vecteurs de mots, TfidfTransformer pour appliquer la pondération TF-IDF, et SGDClassifier comme classificateur linéaire avec descente de gradient stochastique.

In [11]:
# Division des données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(patterns, y, test_size=0.2, random_state=42)

# Création du pipeline avec CountVectorizer, TfidfTransformer et SGDClassifier
pipeline = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', SGDClassifier(loss='hinge', penalty='l2',
                          alpha=1e-3, random_state=42,
                          max_iter=1000, tol=1e-3)),
])

## ENTRAINEMENT ET EVALUATION DU MODELE

Le modèle est entraîné avec pipeline.fit(X, y) sur les données d'entraînement.

In [12]:
# Entraînement du modèle
pipeline.fit(X_train, y_train)

Pipeline(steps=[('vect', CountVectorizer()), ('tfidf', TfidfTransformer()),
                ('clf', SGDClassifier(alpha=0.001, random_state=42))])

In [13]:
# Évaluation du modèle
y_pred_train = pipeline.predict(X_train)
accuracy_train = accuracy_score(y_train, y_pred_train)
print(f'Précision avec données entraînement: {accuracy_train}')

Précision avec données entraînement: 0.9983416252072969


In [14]:
y_pred_test = pipeline.predict(X_test)
accuracy_test = accuracy_score(y_test, y_pred_test)
print(f'Précision avec données test: {accuracy_test}')

Précision avec données test: 0.9823399558498896


In [50]:
# Fonction pour prédire la classe
def predict_class(text):
    predicted = pipeline.predict([text])
    return label_encoder.inverse_transform(predicted)[0]

In [51]:
# Fonction pour obtenir la réponse
def get_response(tag):
    return random.choice(responses[tag])

In [17]:
from sklearn.metrics import classification_report
cr = classification_report(y_test, y_pred_test)
print(cr)

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        10
           2       1.00      1.00      1.00        16
           3       1.00      1.00      1.00        14
           4       1.00      1.00      1.00        15
           5       1.00      1.00      1.00        24
           6       1.00      1.00      1.00        10
           7       1.00      0.79      0.88        14
           8       0.79      1.00      0.88        11
           9       0.91      1.00      0.95        10
          10       1.00      0.94      0.97        18
          11       1.00      1.00      1.00        12
          12       1.00      0.93      0.97        15
          13       0.00      0.00      0.00         2
          14       1.00      1.00      1.00        17
          15       1.00      1.00      1.00         8
          16       1.00      0.94      0.97        18
          17       1.00      1.00      1.00         6
          18       1.00    

/home/ariel/PycharmProjects/enronmails/venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/ariel/PycharmProjects/enronmails/venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/ariel/PycharmProjects/enronmails/venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, mo

## AFFICHAGE GRAPHIQUE 

In [52]:
import tkinter
from tkinter import *
from tkinter import scrolledtext
from tkinter import messagebox
import random

In [53]:
# Interface Utilisateur avec Tkinter
def send():
    question = EntryBox.get("1.0",'end-1c').strip()
    EntryBox.delete("0.0",END)

    if question != '':
        ChatBox.config(state=NORMAL)
        ChatBox.insert(END, "Moi: " + question + '\n\n')
        ChatBox.config(foreground="#446665", font=("Verdana", 12))
    
        tag = predict_class(question)
        response = get_response(tag)
        
        ChatBox.insert(END, "K-ARL: " + response + '\n\n')
        
        ChatBox.config(state=DISABLED)
        ChatBox.yview(END)

In [55]:
root = Tk()
root.title("K-ARL CHATBOT")
root.geometry("400x500")
root.resizable(width=FALSE, height=FALSE)

# Création de la zone de chat
ChatBox = Text(root, bd=0, bg="white", height="8", width="50", font="Arial")
ChatBox.config(state=DISABLED)

# Ajout de la barre de défilement
scrollbar = Scrollbar(root, command=ChatBox.yview, cursor="heart")
ChatBox['yscrollcommand'] = scrollbar.set

# Création du bouton "Envoyer"
SendButton = Button(root, font=("Verdana", 12, 'bold'), text="Send", width="12", height=5,
                    bd=0, bg="#f9a602", activebackground="#3c9d9b", fg='#000000',
                    command=send)

# Création de la zone de saisie
EntryBox = Text(root, bd=0, bg="white", width="29", height="5", font="Arial")

# Placement des composants sur l'écran
scrollbar.place(x=376, y=6, height=386)
ChatBox.place(x=6, y=6, height=386, width=370)
EntryBox.place(x=128, y=401, height=90, width=265)
SendButton.place(x=6, y=401, height=90)

root.mainloop()